In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# 공통 요청 파라미터 설정
KEY = '7a50474368636f64363463436c436f'
TYPE = 'json'
START_INDEX = 1
END_INDEX = 1000

# 시작 날짜 설정
start_date = datetime.strptime('2020-06-01', '%Y-%m-%d')

# 오늘 날짜까지의 데이터 가져오기
end_date = datetime.now().date()

# 모기 데이터 가져오기
SERVICE = 'MosquitoStatus'

# 데이터를 저장할 빈 DataFrame 생성
dfs = []

# 시작 날짜부터 오늘까지의 데이터를 하루씩 가져와서 DataFrame에 추가
while start_date.date() <= end_date:
    MOSQUITO_DATE = start_date.strftime('%Y-%m-%d')
    url = f"http://openapi.seoul.go.kr:8088/{KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{MOSQUITO_DATE}/"
    response = requests.get(url)
    data = response.json()
    try:
        items = data[SERVICE]['row']
        dfs.append(pd.DataFrame(items))
    except KeyError:
        print(f"No data available for date: {MOSQUITO_DATE}")
    start_date += timedelta(days=1)

# 모든 DataFrame을 병합
if dfs:
    df = pd.concat(dfs)

    # 필요한 컬럼 선택
    selected_df = df[['MOSQUITO_DATE', 'MOSQUITO_VALUE_WATER', 'MOSQUITO_VALUE_HOUSE', 'MOSQUITO_VALUE_PARK']]

    # 날짜 형식 변경 및 월별 평균 구하기
    selected_df['YEAR'] = selected_df['MOSQUITO_DATE'].str[:4]
    selected_df['MONTH'] = selected_df['MOSQUITO_DATE'].str[5:7]

    # 'MOSQUITO_VALUE_PARK'에서 숫자가 아닌 문자 제거
    selected_df['MOSQUITO_VALUE_PARK'] = selected_df['MOSQUITO_VALUE_PARK'].str.replace('[^\d.]', '', regex=True)

    # 'MOSQUITO_VALUE_PARK'를 숫자로 변환
    selected_df['MOSQUITO_VALUE_PARK'] = pd.to_numeric(selected_df['MOSQUITO_VALUE_PARK'])

    # 월별 평균 계산
    aggregated_df = selected_df.groupby(['YEAR', 'MONTH'])['MOSQUITO_VALUE_PARK'].mean().reset_index()

    # CSV 파일 저장
    save_path = 'C:\\Users\\user\\Desktop\\2023-1\\data-analysis\\mosquito_data2.csv'
    aggregated_df.to_csv(save_path, index=False)
    print("데이터 저장 완료")
else:
    print("지정된 기간에 대한 데이터가 없습니다.")
